In [1]:
import requests
import json
import pandas as pd
import time
#import convex as cx
import tmqa24 as tmqa

Autosaving every 180 seconds





Using TensorFlow backend.


Loading the params file
Input encoding {'o': 2, '{': 3, '.': 4, 'J': 5, '0': 6, '1': 7, '<': 8, 'B': 9, 'd': 10, '£': 11, 'e': 12, '6': 13, '!': 14, 'O': 15, 'M': 16, 'X': 17, 'f': 18, 't': 19, 'C': 20, 'V': 21, 'z': 22, 'K': 23, '\\': 24, '9': 25, 'P': 26, 'S': 27, '/': 28, '₹': 29, 'F': 30, 'G': 31, '=': 32, '8': 33, ')': 34, '+': 35, ']': 36, 'U': 37, "'": 38, '"': 39, 'g': 40, 'N': 41, 'r': 42, 'u': 43, '&': 44, '$': 45, 'x': 46, '%': 47, ':': 48, '@': 49, '^': 50, 'I': 51, 'L': 52, 'Z': 53, 'h': 54, 'W': 55, 'A': 56, 'v': 57, '?': 58, '2': 59, '~': 60, 's': 61, 'T': 62, 'R': 63, ',': 64, '|': 65, '4': 66, '>': 67, 'y': 68, '(': 69, '[': 70, 'k': 71, 'H': 72, 'l': 73, 'j': 74, '7': 75, 'n': 76, 'i': 77, 'D': 78, 'Q': 79, ' ': 80, 'm': 81, 'Y': 82, '*': 83, '}': 84, '#': 85, 'p': 86, 'q': 87, '5': 88, 'c': 89, '`': 90, 'a': 91, 'b': 92, 'w': 93, '3': 94, 'E': 95, ';': 96, '-': 97}
Input decoding {2: 'o', 3: '{', 4: '.', 5: 'J', 6: '0', 7: '1', 8: '<', 9: 'B', 10: 'd', 11: '£', 12: '

In [2]:
#f_test = open("/data/users/romain.claret/tm/wikidata-simplequestions/annotated_wd_data_test.txt",'r')
#out_test = f_test.readlines()

In [3]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [4]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    #if (query.json()['questions'][0]['question']['answers'].replace('\n', '')) == None:
    #    return False
    #print(query.json()['questions'][0]['question']['answers'].replace('\n', '').get("results"))
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")
#ask_qanswer("was the film helpmates in color or black-and-white?")

In [5]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")
#ask_platypus("was the film helpmates in color or black-and-white?")
#ask_platypus("Who created the show life on earth")
#ask_platypus("Who is the wife of Barack Obama")

In [6]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    try:
        if not cx_result:
            return cx_result
        if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
            return False
        return cx_result
    except:
        return False

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_convex("Who is the wife of Barack Obama")
#ask_convex("100% senorita is a television show in what language?")

In [7]:
def ask_tmqa(question, verbose=False, timer=False, show_graph=False, cores=tmqa.mp.cpu_count(), banning_str=False):
    tmqa_result = tmqa.answer_question(question, verbose=verbose, timer=timer, show_graph=show_graph, cores=cores, banning_str=banning_str)
    #print(tmqa_result)
    if not tmqa_result:
        return tmqa_result
    #if tmqa_result[]
    #if tmqa_result[0][0][:1] != 'P' and tmqa_result[0][0][:1] != 'Q':
    #    return False
    return tmqa_result[0]

#answer = ask_tmqa_1("Which actor voiced the Unicorn in The Last Unicorn?", verbose=True)
#answer = ask_tmqa_1("what's akbar tandjung's ethnicity", verbose=True)
#ask_tmqa_1("Which genre of album is harder.....faster?")
#ask_tmqa_1("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#answer = ask_tmqa("Who is the wife of Barack Obama")
#print(answer)

In [8]:
# Evaluate qanswer
#arguments[0] Subject
#arguments[1] Predicate
#arguments[2] Object
#arguments[3] question

#HEADERS = ['question', 'source', 'qanswer', 'platypus', 'convex', 'tmqa']
#df = pd.DataFrame(columns=HEADERS)

#start_time = time.time()
#out_test_len = len(out_test)
##print(out_test_len)
#for i, line in enumerate(out_test):
#    arguments = line.split("\t")
#    print("\r\t>>> Processing {}/{}: {}".format(i,out_test_len,arguments[3][:-1]), end='                                  ')
#    if i < len(df):
#        continue
#    result_qanswer = ask_qanswer(arguments[3][:-1])
#    result_platypus = ask_platypus(arguments[3][:-1])
#    result_convex = ask_convex(arguments[3][:-1])
#    #result_tmqa = ask_tmqa(arguments[3][:-1])[1]
#    print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == arguments[2] else False
#    print("\n",i, "-> Platypus", result_platypus) if result_platypus == arguments[2] else False
#    print("\n",i, "-> Convex", result_convex) if result_convex == arguments[2] else False
#    #print("\n",i, "-> TMqa", result_tmqa) if result_tmqa == arguments[2] else False
#    df = df.append({"question":arguments[3][:-1], 'source':arguments[2], 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex, 'tmqa1':result_tmqa}, ignore_index=True)
#
#end_time = time.time()
#print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
#print(df)

In [9]:
#len(df)

In [10]:
#df.tail(5)

In [11]:
#SAVING
#pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(len(df_loaded)))

In [12]:
#LOADING
#df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-9961.pickle.bz2")

#/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-5.pickle.bz2


In [13]:
#df = df_loaded

In [14]:
#df_loaded

In [15]:
#df_loaded = df_loaded.replace("", False)

In [16]:
#df_loaded['qanswer'][34] = False

In [17]:
#del df_loaded['tm2']

In [18]:
#df_loaded.rename({'mine':'tm1'}, axis=1, inplace=True)

In [19]:
#df_loaded['tm1_top4'] = "False"

In [20]:
#df_loaded = df_loaded[['question','source','qanswer','platypus','convex','tm1','tm1_time','tm1_top2','tm1_top3','tm1_top4','tm1_top5','tm1_topall']]

In [21]:
#df_loaded

In [22]:
#LOADING QAGraph

df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-193.pickle.bz2")



In [23]:
print(df_loaded.dtypes)

question      object
source        object
qanswer       object
platypus      object
convex        object
tm1           object
tm1_time      object
tm1_top2      object
tm1_top3      object
tm1_top4      object
tm1_top5      object
tm1_topall    object
dtype: object


In [ ]:
banning_str = [["ř","r"],["ø","o"]]

df_loaded_len = len(df_loaded)
for i, question in enumerate(df_loaded['question']):
    if i >= 272:
    #if i >= 497:
        source = str(df_loaded['source'][i])
        print(str(i)+"/"+str(df_loaded_len),question,"-> source:",source)
        
        start_time = time.time()
        result_tmqa = ask_tmqa(question, verbose=True, timer=True, banning_str=banning_str)
        
        if result_tmqa:
            df_loaded['tm1'][i] = result_tmqa[0]
            if source in result_tmqa[:2]:
                df_loaded['tm1_top2'][i] = True
            if source in result_tmqa[:3]:
                df_loaded['tm1_top3'][i] = True
            if source in result_tmqa[:4]:
                df_loaded['tm1_top4'][i] = True
            if source in result_tmqa[:5]:
                df_loaded['tm1_top5'][i] = True
            if source in result_tmqa:
                df_loaded['tm1_topall'][i] = True
        else:
            df_loaded['tm1'][i] = False
        end_time = time.time()
        df_loaded['tm1_time'][i] = round(end_time-start_time,2)
        print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
        print(str(str(df_loaded['tm1'][i])==str(source)),"---> result_tmqa:",str(source))
        
        pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(i))
        
        print(df_loaded.loc[i,:])
        print("\n")

272/9961 what is the genre of bernthøler? -> source: Q37073
User input: what is the genre of bernthøler?
--> Auto correcting question in progress...
-> Auto corrected q_nlp: What is the genre of Berntholer 
-> q_themes: ([(genre, ['Q483394', 'Q5533543', 'P136'])], [is the genre of Berntholer])
-> q_themes_enhanced: [('Genre', ['Q28490034', 'P136'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
-> q_predicates: [(be, ['P31']), (genre, ['P136', 'P415'])]
->	Running time is 4.82s
-> q_focused_parts: [(genre, ['P136', 'Q483394'])]
-> Building the graph with k_deep 2 ... (could be long)
->	Running time is 15.67s
-->  323 nodes and 320 edges
--> Removing meaningless subgraphs
--> New graph of: 320 nodes and 318 edges
-> predicates_dict: {'P136': 3, 'P1433': 1, 'P31': 150, 'P460': 1, 'P360': 1, 'P407': 1, 'P1963': 1, 'P642': 1, 'P495': 1}
-> paths_keywords: (['genre', 'is the genre of berntholer'], {'Genre': [genre, ['P136']]}, [What])
-> Computing pos

->	Running time is 13.46s
-->  80 nodes and 80 edges
--> Removing meaningless subgraphs
--> New graph of: 80 nodes and 80 edges
---> Loop detected, returning the graph in the current state
-> predicates_dict: {'P527': 1, 'P735': 1, 'P20': 1, 'P19': 2, 'P509': 1, 'P1196': 1, 'P31': 5, 'P361': 3, 'P1552': 1, 'P1269': 3, 'P279': 1, 'P577': 1, 'P4934': 2, 'P156': 2, 'P155': 2, 'P27': 1, 'P136': 3, 'P264': 2, 'P569': 1, 'P21': 1, 'P106': 2, 'P175': 1, 'P547': 1, 'P570': 1}
-> paths_keywords: (['hank cochran'], {}, [Where])
-> Computing possible paths... (could be long)
--> len(path_nodes): 222
->	Running time is 15.98s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 178
->	Running time is 2.57s
-> Computing hypothesises...
date_trigger False
--> hypothesises: [['Q23197', 3.7208280118029164], ['Q7129202', 1.5264340648242125], ['Q2389905', 0.6710912092622235], ['Q3016939', 0.6544586100312213], ['Q21821348', 0.57047901991415], ['Q47223', 0.4845811652335261], ['Q55090886', 

question      What is the name of a music program 
source                                   Q16836120
qanswer                                      False
platypus                                     False
convex                                   Q10590618
tm1                                        Q783521
tm1_time                                    189.31
tm1_top2                                     False
tm1_top3                                     False
tm1_top4                                     False
tm1_top5                                     False
tm1_topall                                   False
Name: 276, dtype: object


277/9961 who played the bass guitar in Communic -> source: Q16450514
User input: who played the bass guitar in Communic
--> Auto correcting question in progress...
-> Auto corrected q_nlp: Who played the bass guitar in Communic 
-> q_themes: ([(Communic, ['Q1120408']), (bass guitar, ['Q46185', 'Q4867919'])], [the bass guitar, played the bass guitar in Communi

--> hypothesises: [['Q4611891', 0.761154341547137], ['Q1151733', 0.690361143175657], ['Q1148974', 0.6707160353853207], ['Q18533990', 0.6277843512376072], ['Q28648649', 0.6162048328726359], ['Position (team sports)', 0.5196061486812047]]
->	Running time is 8.68s
-> Computing golden paths...
hypo_diff 0.07079319837148002
first_is_winner False
--> len(golden_paths): 1
->	Running time is 3.11s
--> len(cleared_golden_paths): 1
---> First path: ['association football position', 'subclass of', 'position', 'Commons category', 'Position (team sports)']
->	Total Running time is 144.6s
->	Running time is 144.78s
False ---> result_tmqa: Q280658
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-278.pickle.bz2
question      Which position is played by guillermo barros s...
source                                                  Q280658
qanswer                                                 Q170703
platypus          

->	Running time is 25.54s
-->  149 nodes and 140 edges
--> Removing meaningless subgraphs
--> New graph of: 149 nodes and 140 edges
-> predicates_dict: {'P1013': 1, 'P106': 4, 'P279': 2, 'P175': 1, 'P101': 3, 'P241': 1, 'P734': 2, 'P31': 7, 'P607': 12, 'P3744': 1, 'P407': 1, 'P3984': 1, 'P138': 3, 'P527': 1, 'P361': 1, 'P102': 2, 'P580': 1, 'P582': 1, 'P1344': 1, 'P360': 1, 'P495': 1, 'P425': 2, 'P571': 1, 'P373': 2, 'P739': 1, 'P39': 1, 'P287': 1, 'P1225': 1, 'P452': 1, 'P123': 1, 'P17': 1, 'P910': 2, 'P282': 1, 'P856': 1, 'P460': 1, 'P50': 1, 'P1080': 1, 'P5429': 2, 'P3879': 1}
-> paths_keywords: (['involved', 'liberators', 'liberators civil war', 'civil war'], {'employees': [employees, ['P1128']], 'personnel': [employees, ['P1128']]}, [])
-> Computing possible paths... (could be long)
--> len(path_nodes): 244
->	Running time is 9.85s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 234
->	Running time is 2.54s
-> Computing hypothesises...
date_trigger False
--> h

...

-> Auto corrected q_nlp: What gender is Laura kirkpatrick 
-> q_themes: ([(Laura kirkpatrick, ['Q1935027']), (Gender, ['Q48277', 'Q15753030', 'Q10503768', 'Q19122628']), (gender, ['Q5531000'])], [What gender is Laura])
-> q_themes_enhanced: []
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
-> q_predicates: [(be, ['P31']), (gender, ['P21'])]
->	Running time is 4.87s
-> q_focused_parts: [(gender, ['Q5531000', 'P21'])]
-> Building the graph with k_deep 2 ... (could be long)
->	Running time is 10.13s
-->  58 nodes and 54 edges
--> Removing meaningless subgraphs
--> New graph of: 58 nodes and 54 edges
---> Rebuilding the graph with k_deep 4 ... Previously: 58 nodes or 54 edges was below the limit of 100
->	Running time is 10.61s
-->  69 nodes and 66 edges
--> Removing meaningless subgraphs
--> New graph of: 69 nodes and 66 edges
---> Rebuilding the graph with k_deep 6 ... Previously: 69 nodes or 66 edges was below the limit of 100
->	Running time is 10

-> Auto corrected q_nlp: What gameplay options are available for the game growlanser I'v wayfarer of the time 
-> q_themes: ([(the game, ['Q211405', 'Q723187']), (the time, ['Q1571747', 'Q3989647', 'Q2414659']), (The Game, ['Q1170403']), (The Time, ['Q398689']), (time, ['Q43297', 'Q11471', 'Q493981'])], [gameplay options, growlanser I'v wayfarer, Gameplay Options, Growlanser I'v Wayfarer, growlanser i'v wayfarer, gameplay option, game growlanser, I'v wayfarer])
-> q_themes_enhanced: [('gameplay', ['Q1331296']), ('option', ['Q187860', 'Q187860']), ('Gameplay', ['Q2720327']), ('Options', ['Q28453077']), ('Option', ['Q175229']), ('Growlanser', ['Q5611687']), ('Wayfarer', ['Q1798950'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: wayfarer
-> q_predicates: [(be, ['P31']), (I'v, ['P131', 'P2553']), (gameplay, ['P404']), (options, ['P4837']), (available, ['P407', 'P953']), (game, ['P1350', 'P400']), (wayfarer, []

-> Auto corrected q_nlp: Who wrote the film the inner circle 
-> q_themes: ([(the film, ['Q25302710', 'Q3520871', 'Q16679544']), (The Inner Circle, ['Q2378497', 'Q2732154', 'Q2412108', 'Q16614322', 'Q16251846', 'Q19364490'])], [])
-> q_themes_enhanced: []
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
-> q_predicates: [(wrote, ['P1412']), (film, ['P136', 'P57']), (inner, ['P3189', 'P3190']), (circle, ['P1776', 'P3455'])]
->	Running time is 5.23s
-> q_focused_parts: [(film, ['Q6293', 'Q11424']), (circle, ['Q238231', 'Q17278', 'Q2013644', 'Q5121385'])]
-> Building the graph with k_deep 2 ... (could be long)
->	Running time is 11.82s
-->  73 nodes and 72 edges
--> Removing meaningless subgraphs
--> New graph of: 68 nodes and 68 edges
---> Rebuilding the graph with k_deep 4 ... Previously: 68 nodes or 68 edges was below the limit of 100
->	Running time is 12.72s
-->  102 nodes and 104 edges
--> Removing meaningless subgraphs
--> New graph of: 95 nodes

->	Running time is 8.29s
-->  37 nodes and 38 edges
--> Removing meaningless subgraphs
--> New graph of: 26 nodes and 28 edges
---> Loop detected, returning the graph in the current state
-> predicates_dict: {'P407': 1, 'P735': 1, 'P19': 1, 'P31': 2, 'P569': 1, 'P577': 1, 'P27': 1, 'P136': 1, 'P734': 1, 'P106': 2, 'P21': 1, 'P50': 1, 'P162': 1, 'P57': 2, 'P58': 2}
-> paths_keywords: (['rob margolies', 'birthplace', 'the birthplace'], {}, [Where])
-> Computing possible paths... (could be long)
--> len(path_nodes): 30
->	Running time is 16.2s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 22
->	Running time is 2.55s
-> Computing hypothesises...
date_trigger False
--> hypothesises: [['Q1087523', 16.902805864585034], ['Q30', 3.04875043194891], ['Q5', 0.4902600215256651], ['1983-02-28T00:00:00Z', 0.19270498704622788]]
->	Running time is 4.65s
-> Computing golden paths...
hypo_diff 13.854055432636123
first_is_winner True
--> len(golden_paths): 4
->	Running time is 0.0s


--> len(golden_paths): 3
->	Running time is 2.83s
--> len(cleared_golden_paths): 3
---> First path: ['Vladimir Seleznev', 'genre', 'Genre', 'said to be the same as', 'art genre']
->	Total Running time is 111.34s
->	Running time is 111.53s
False ---> result_tmqa: Q289
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-296.pickle.bz2
question      what is the genre of the amazing mrs pritchard
source                                                  Q289
qanswer                                             Q1366112
platypus                                               False
convex                                              Q1366112
tm1                                                 Q4413495
tm1_time                                              111.53
tm1_top2                                               False
tm1_top3                                               False
tm1_top4                          

In [ ]:
df_loaded.head(10)

In [ ]:
df = df_loaded.copy()
#df = df.replace("", False)

In [ ]:
max_row = 150
df_len = len(df)
df_qanswer_max = df[(df.index<=max_row) & (df.qanswer == df.source)]
df_qanswer_max_len = len(df_qanswer_max)

df_platypus_max = df[(df.index<=max_row) & (df.platypus == df.source)]
df_platypus_max_len = len(df_platypus_max)

df_convex_max = df[(df.index<=max_row) & (df.convex == df.source)]
df_convex_max_len = len(df_convex_max)

df_tm1_max = df[(df.index<=max_row) & (df.tm1 == df.source)]
df_tm1_max_len = len(df_tm1_max)

df_tm1_max_top2 = df[(df.index<=max_row) & (df.tm1_top2 == True)]
df_tm1_max_top2_len = len(df_tm1_max_top2)

df_tm1_max_top3 = df[(df.index<=max_row) & (df.tm1_top3 == True)]
df_tm1_max_top3_len = len(df_tm1_max_top3)

df_tm1_max_top4 = df[(df.index<=max_row) & (df.tm1_top4 == True)]
df_tm1_max_top4_len = len(df_tm1_max_top4)

df_tm1_max_top5 = df[(df.index<=max_row) & (df.tm1_top5 == True)]
df_tm1_max_top5_len = len(df_tm1_max_top5)

df_tm1_max_topall = df[(df.index<=max_row) & (df.tm1_topall == True)]
df_tm1_max_topall_len = len(df_tm1_max_topall)

print("qanswer:", df_qanswer_max_len,df_qanswer_max_len/max_row)
print("platypus:", df_platypus_max_len, df_platypus_max_len/max_row)
print("convex:", df_convex_max_len, df_convex_max_len/max_row)
print("tm1:", df_tm1_max_len, df_tm1_max_len/max_row)
print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)

#df[ & (df.qanswer == df.source)]

In [ ]:
print("qanswer:",len(df[df.qanswer == df.source]),len(df[df.qanswer == df.source])/len(df))
print("platypus:",len(df[df.platypus == df.source]),len(df[df.platypus == df.source])/len(df))
print("convex:",len(df[df.convex == df.source]),len(df[df.convex == df.source])/len(df))
print("tm1:",len(df[df.tm1 == df.source]),len(df[df.tm1 == df.source])/len(df))
print("tm1_top2:",len(df[df.tm1_top2 == True]),len(df[df.tm1_top2 == True])/len(df))
print("tm1_top3:",len(df[df.tm1_top3 == True]),len(df[df.tm1_top3 == True])/len(df))
print("tm1_top4:",len(df[df.tm1_top4 == True]),len(df[df.tm1_top4 == True])/len(df))
print("tm1_top5:",len(df[df.tm1_top5 == True]),len(df[df.tm1_top5 == True])/len(df))
print("tm1_topall:",len(df[df.tm1_topall == True]),len(df[df.tm1_topall == True])/len(df))

In [ ]:
#df.tm1_top2 = df.tm1_top3